# Scenario models

For a delta-change in a feature (or group of features), what is the likely change in displacement?

### Assumptions:
- changes and impacts occur in year=0, year=1
- changes across themes are additive
- User-specified change are a percentage change. 
- 3 geographies - 'sub-global', 'AFG', 'MMR'
  
### Notes
- groupings.json - need to be verified with DRC.

- Sub-global is a set of countries follow the critiera: "Simply look at major displacement countries in 2018 and pick the major ones including those with more recent displacement, which share some similar features in the displacement nature and with DRC presence/operations (e.g. we leave out Central American countries because displacement is unlike most other places in the world driven by crime and we do not have any operational presence, nor planning one)"

In [1]:
subglobal = ['SYR','COL','AFG','COG','SSD','SOM','VEN','ETH','SDN','NGA',
             'IRQ','YEM','UKR','MMR','CAF','CMR','ERI','BDI','GEO','MLI',
             'TCD','LBY','NER','BFA','COD']

In [2]:
import pandas as pd
from time import time
import os
import json
import pickle
import numpy as np 
from time import time
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

# Data transforms

<TBC>

In [3]:
start_time = time()
with open("../configuration.json", 'rt') as infile:
    config = json.load(infile)


sources = [os.path.join("..", config['paths']['output'], 
                        d['name'], 
                        'data.csv') for d in config['sources'] if d['name']]


# Generate a data frame with all indicators
df = pd.concat((pd.read_csv(f) for f in sources), sort=False, ignore_index=True)

# Summary stats
print("Sources            : {}".format(len(sources)))
print("Shape              : {} (rows) {} (columns)".format(*df.shape))
print("Geographies        : {}".format(len(df['Country Name'].unique())))
print("Indicators         : {}".format(len(df['Indicator Code'].unique())))
print("Temporal coverage  : {} -> {}".format(df.year.min(), df.year.max()))
print("Null values        : {}".format(sum(df['value'].isnull())))

print("\nLoaded data in {:3.2f} sec.".format(time() - start_time))


# Now arrange data in wide form
data = pd.pivot_table(df, index=['Country Code', 'year'],
                      columns='Indicator Code', values='value')

# Consider country/year as features (and not an index)
data.reset_index(inplace=True)

print("Long form of size  : {} (rows) {} (columns)".format(*data.shape))

Sources            : 19
Shape              : 924558 (rows) 6 (columns)
Geographies        : 568
Indicators         : 142
Temporal coverage  : 1789 -> 2019
Null values        : 1421

Loaded data in 1.12 sec.
Long form of size  : 35901 (rows) 144 (columns)


In [4]:
# Only look at features used in scenarios
groupings = json.load(open("../groupings.json", 'rt'))
featureset = [i['code'] for c in groupings['clusters'] for i in c['indicators']]

# Dimensions of interest
CLUSTERS = groupings['clusters']
COUNTRIES = ['subglobal', 'AFG', 'MMR']
LAGS = [0, 1]

def lag_variables(data, var, lag):
    """
    Append lagged variables to frame.
    
    data - pandas data frame
    var - list of variable names to lag
    lag - integer
    
    """
    idx_cols = ['year', 'Country Code']
    fv = var + idx_cols
    
    tmp = data[fv].copy(deep=True)
    
    col_name = [v + ".T" + "{0:+}".format(lag) for v in var]
    
    tmp.rename(columns={k: v for (k, v) in zip(var, col_name)},
               inplace=True)
    tmp.year -= lag
    data = pd.merge(data, tmp, on=idx_cols, how='left')
    
    return data, col_name


def model_case(data, lag, countries):
    """ Generate a data frame to estimate elasticities """
    
    # Features
    idx = ['Country Code', 'year']
    TARGETS = ['DRC.TOT.DISP']

    # spatial filter
    if countries.lower() != 'subglobal':
        if ~isinstance(countries, list):
            countries = [countries]
        c1 = data['Country Code'].isin(countries)
    else:
        # use DRC list of countries
        c1 = data['Country Code'].isin(subglobal)
    
    if lag != 0:
        df, TARGETS = lag_variables(data, TARGETS, lag)
    else:
        df = data.copy(deep=True)
        
    # temporal filter
    c2 = data.year >= 1980

    df = df.loc[c1 & c2, idx + featureset + TARGETS]
    
    X = df.loc[:, featureset]
    Y = df.loc[:, TARGETS]
    
    # handle missing labels
    idx = ~pd.isnull(Y.values)
    Y = Y[idx]
    X = X[idx]
    
    # Missing data
    X.fillna(X.mean(), inplace=True)
    Y.fillna(Y.mean(), inplace=True)

    return X, Y

In [5]:

for c in ['AFG']:
    for v in featureset:
        c1 = data['Country Code'] == c

        Y = data.loc[c1, 'DRC.TOT.DISP']
        X = data.loc[c1, v]

        # handle missing labels
        idx = ~pd.isnull(Y.values)
        Y = Y[idx]
        X = X[idx]

        # Missing data
        X.fillna(X.mean(), inplace=True)
        Y.fillna(Y.mean(), inplace=True)
        X1 = sm.add_constant(X)

        m = sm.OLS(Y, X1)
        res = m.fit()
        
        fig = plt.figure(figsize=(12,8))
        fig = sm.graphics.plot_regress_exog(res, v, fig=fig)
        plt.savefig("img/regplots/{}-{}.png".format(c, v))
        plt.close()



/Users/rahul/anaconda3/envs/mm4sight/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


# Assemble and build

In [6]:
from itertools import product

models = {}
for lag, c in product(LAGS, COUNTRIES):
    
    X, Y = model_case(data, lag, c)
    
    X1 = sm.add_constant(X)

    # Pick a random validation sample
    Xv = X1.head(1)
    
    M = {}
    key = c, lag
    clf = sm.OLS(Y, X1)
    m = clf.fit()
    M['significance'] = (m.pvalues<0.05).to_dict()
    M['elasticity'] = m.params.to_dict()
    M['model'] = m
    M['Xv'] = X1.head(1)
    models[key] = M


In [7]:
theme_indicator_map = {c['theme']: c['indicators'] for c in CLUSTERS}

def get_significance(pvalues, indicators):
    
    c = [i['code'] for i in indicators]
    s = [pvalues[i] for i in c]

    if all(s):
        # Statistically significant for entire cluster
        return "*"
    elif any(s):
        # only some indicators within cluster are statistically significant
        return "x"
    else:
        # Not statistically significant
        return "ns"
    
def elasticity(M, scenario):
    """
    Compute elasticity of target variable relative to a 
    user-specified scenario.
    M - linear models with variable level groupings
    scenario - dictionary of scenario inputs
    
    returns change in displacement
    """

    e = M['elasticity']
    m = M['model']
    sig = M['significance']
    Xv = M['Xv']
    
    bl = m.predict(Xv).values[0]
    # Numeric equivalent
    user_scenario = {}

    for theme, change in scenario.items():

        # get the percentage change set
        if change == 'NC':
            ds = 0.0
        else:
            ds = float(change.replace('%', '')) / 100.0
        indicatorset = theme_indicator_map[theme]

        # Is the change statistically significant
        significance = get_significance(sig, indicatorset)
        
        for i in indicatorset:
            if i['direction-improvement'] == 'lower':
                # flip the direction
                user_scenario[i['code']] = -1 * ds
            else:
                user_scenario[i['code']] =  ds

    # numerical change
    num_change = {k: v * np.abs(Xv[k].values[0]) for k, v in user_scenario.items()}
    b = Xv.to_dict(orient='records')[0]
    for k, v in num_change.items():
        if ~(np.abs(v - b[k]) <= 1e-5):
            verb = 'reduced' if v <=0 else 'increased'
                
            #print("{} {} from {:.1f} by {:.1f} (significant? {}) dt: {:.0f}, e: {:.0f}".format(k, verb, b[k], v, sig[k], e[k] * v, e[k]))
    
    ind_dt = [e[k] * v for k, v in num_change.items()]

    # finally report change relative to baseline
    dt = sum(ind_dt)/bl
    return dt, significance
    # verb = 'increase' if dt > 0 else 'decrease'
    #print("{} change results in {:.2%} {} in forced displacement ({}).".format(scenario, np.abs(dt), verb, significance))



In [8]:
result = []
for cl, country in product(CLUSTERS, COUNTRIES):
    for l in cl['labels']:
    
        lag = 0
        scenario = {cl['theme']: l}
        key = (country, lag)
        M = models[key]

        dt, sig = elasticity(M, scenario)

        result.append({
            'theme': cl['theme'],
            'change-ind': l,
            'country': country,
            'lag': lag,
            'change-disp': dt,
            'significance': sig
        })

In [13]:
rdf = pd.DataFrame(result)
rdf['value'] = rdf.apply(lambda x: "{:+1.2f}% {}".format(100*x['change-disp'], x['significance']), axis=1)
rdf.head()

,theme,change-ind,country,lag,change-disp,significance,value
0,Economy,-10%,subglobal,0,-0.023916,x,-2.39% x
1,Economy,-5%,subglobal,0,-0.011958,x,-1.20% x
2,Economy,NC,subglobal,0,0.000000,x,+0.00% x
3,Economy,+5%,subglobal,0,0.011958,x,+1.20% x
4,Economy,+10%,subglobal,0,0.023916,x,+2.39% x


In [14]:
#rdf.set_index(['country', 'theme', 'lag'])

tmp = rdf[rdf['lag']== 0]

tmp = pd.pivot_table(rdf, index=['country', 'theme', 'lag'], 
                     columns='change-ind', values='value',
                    aggfunc=lambda x: ' '.join(x)) 

tmp

change-ind                       +10%         +2%          +4%         +5%  \
country   theme      lag                                                     
AFG       Conflict   0      -11.07% x         NaN          NaN    -5.53% x   
          Economy    0       +2.68% x         NaN          NaN    +1.34% x   
          Governance 0            NaN   -11.87% x    -23.75% x         NaN   
          Natural    0            NaN   +6.44% ns   +12.88% ns         NaN   
          Population 0            NaN   +0.05% ns    +0.11% ns         NaN   
MMR       Conflict   0     -15.88% ns         NaN          NaN   -7.94% ns   
          Economy    0    +146.62% ns         NaN          NaN  +73.31% ns   
          Governance 0            NaN  -53.81% ns  -107.63% ns         NaN   
          Natural    0            NaN  -29.25% ns   -58.50% ns         NaN   
          Population 0            NaN  -104.44% x   -208.87% x         NaN   
subglobal Conflict   0       -2.72% x         NaN          NaN    -1.36% x   
          Economy    0       +2.39% x         NaN          NaN    +1.20% x   
          Governance 0            NaN    -2.79% x     -5.58% x         NaN   
          Natural    0            NaN    +0.07% x     +0.15% x         NaN   
          Population 0            NaN    +0.04% x     +0.07% x         NaN   

change-ind                       -10%         -2%          -4%         -5%  \
country   theme      lag                                                     
AFG       Conflict   0      +11.07% x         NaN          NaN    +5.53% x   
          Economy    0       -2.68% x         NaN          NaN    -1.34% x   
          Governance 0            NaN   +11.87% x    +23.75% x         NaN   
          Natural    0            NaN   -6.44% ns   -12.88% ns         NaN   
          Population 0            NaN   -0.05% ns    -0.11% ns         NaN   
MMR       Conflict   0     +15.88% ns         NaN          NaN   +7.94% ns   
          Economy    0    -146.62% ns         NaN          NaN  -73.31% ns   
          Governance 0            NaN  +53.81% ns  +107.63% ns         NaN   
          Natural    0            NaN  +29.25% ns   +58.50% ns         NaN   
          Population 0            NaN  +104.44% x   +208.87% x         NaN   
subglobal Conflict   0       +2.72% x         NaN          NaN    +1.36% x   
          Economy    0       -2.39% x         NaN          NaN    -1.20% x   
          Governance 0            NaN    +2.79% x     +5.58% x         NaN   
          Natural    0            NaN    -0.07% x     -0.15% x         NaN   
          Population 0            NaN    -0.04% x     -0.07% x         NaN   

change-ind                       NC  
country   theme      lag             
AFG       Conflict   0     +0.00% x  
          Economy    0     +0.00% x  
          Governance 0     +0.00% x  
          Natural    0    +0.00% ns  
          Population 0    +0.00% ns  
MMR       Conflict   0    -0.00% ns  
          Economy    0    -0.00% ns  
          Governance 0    -0.00% ns  
          Natural    0    -0.00% ns  
          Population 0     -0.00% x  
subglobal Conflict   0     +0.00% x  
          Economy    0     +0.00% x  
          Governance 0     +0.00% x  
          Natural    0     +0.00% x  
          Population 0     +0.00% x

In [11]:
tmp

change-ind                      +10%        +2%         +4%        +5%  \
country   theme      lag                                                 
AFG       Conflict   0      -11.1% x        NaN         NaN    -5.5% x   
          Economy    0       +2.7% x        NaN         NaN    +1.3% x   
          Governance 0           NaN   -11.9% x    -23.7% x        NaN   
          Natural    0           NaN   +6.4% ns   +12.9% ns        NaN   
          Population 0           NaN   +0.1% ns    +0.1% ns        NaN   
MMR       Conflict   0     -15.9% ns        NaN         NaN   -7.9% ns   
          Economy    0    +146.6% ns        NaN         NaN  +73.3% ns   
          Governance 0           NaN  -53.8% ns  -107.6% ns        NaN   
          Natural    0           NaN  -29.2% ns   -58.5% ns        NaN   
          Population 0           NaN  -104.4% x   -208.9% x        NaN   
subglobal Conflict   0       -2.7% x        NaN         NaN    -1.4% x   
          Economy    0       +2.4% x        NaN         NaN    +1.2% x   
          Governance 0           NaN    -2.8% x     -5.6% x        NaN   
          Natural    0           NaN    +0.1% x     +0.1% x        NaN   
          Population 0           NaN    +0.0% x     +0.1% x        NaN   

change-ind                      -10%        -2%         -4%        -5%  \
country   theme      lag                                                 
AFG       Conflict   0      +11.1% x        NaN         NaN    +5.5% x   
          Economy    0       -2.7% x        NaN         NaN    -1.3% x   
          Governance 0           NaN   +11.9% x    +23.7% x        NaN   
          Natural    0           NaN   -6.4% ns   -12.9% ns        NaN   
          Population 0           NaN   -0.1% ns    -0.1% ns        NaN   
MMR       Conflict   0     +15.9% ns        NaN         NaN   +7.9% ns   
          Economy    0    -146.6% ns        NaN         NaN  -73.3% ns   
          Governance 0           NaN  +53.8% ns  +107.6% ns        NaN   
          Natural    0           NaN  +29.2% ns   +58.5% ns        NaN   
          Population 0           NaN  +104.4% x   +208.9% x        NaN   
subglobal Conflict   0       +2.7% x        NaN         NaN    +1.4% x   
          Economy    0       -2.4% x        NaN         NaN    -1.2% x   
          Governance 0           NaN    +2.8% x     +5.6% x        NaN   
          Natural    0           NaN    -0.1% x     -0.1% x        NaN   
          Population 0           NaN    -0.0% x     -0.1% x        NaN   

change-ind                      NC  
country   theme      lag            
AFG       Conflict   0     +0.0% x  
          Economy    0     +0.0% x  
          Governance 0     +0.0% x  
          Natural    0    +0.0% ns  
          Population 0    +0.0% ns  
MMR       Conflict   0    -0.0% ns  
          Economy    0    -0.0% ns  
          Governance 0    -0.0% ns  
          Natural    0    -0.0% ns  
          Population 0     -0.0% x  
subglobal Conflict   0     +0.0% x  
          Economy    0     +0.0% x  
          Governance 0     +0.0% x  
          Natural    0     +0.0% x  
          Population 0     +0.0% x

#### Summary

- Results for Myanmar are poor. Besides population most categories are not statistically significant. The magnitude and signs of changes in displacement suggest that the OLS method is unsuitable for this case. 
- Fall back to the Sub-Global (25 country basket) appears to be a good strategy.
- For AFG, elasticities relative to the thematic clusters make intuitive sense. The lag effects also imply, a decay in conflict impact, and a delayed impact of economy. 
- One problem however is that the scales are different for each thematic cluster. `Governance` indicators for example seem to be short scaled data (1 to 4 or 1 to 10). A 10% change can represent a big change. Cluster-specific labels such that one unit change in theme variable are comparable


In [12]:
# Scale Normalization

-

SyntaxError: invalid syntax (<ipython-input-12-5b9a43575971>, line 3)